In [3]:
import numpy as np
import torch
from sklearn.linear_model import QuantileRegressor

In [5]:
beta = np.random.randn(10)
X = np.random.randn(10000 * 10).reshape(10000, 10)
Y = X @ beta + np.random.randn(10000)

In [15]:
QuantileRegressor(quantile=0.5, alpha=0.0).fit(X, Y)

TypeError: zeros() received an invalid combination of arguments - got (int, type=type, requires_grad=bool), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [6]:
Xt = torch.as_tensor(X)
Yt = torch.as_tensor(Y)

In [112]:
beta_hat = torch.tensor(np.zeros(10), requires_grad=True)
beta0_hat = torch.tensor(0.0, requires_grad=True)

In [113]:
def quantile_loss(error, q):
    loss = q * torch.relu(error) + (1 - q) * torch.relu(-error)
    return loss

In [114]:
q = 0.8

In [115]:
quantile_loss(Yt - Xt @ beta_hat - beta0_hat, 0.8).mean()

tensor(1.1472, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [116]:
L = max(q, 1 - q)

optim = torch.optim.SGD([beta_hat, beta0_hat], lr=1 / L)

for i in range(20):
    loss = quantile_loss(Yt - Xt @ beta_hat - beta0_hat, q).mean()
    loss.backward()
    optim.step()
    optim.zero_grad()

In [117]:
loss

tensor(0.2799, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [118]:
(Yt - Xt @ beta_hat - beta0_hat > 0).float().mean()

tensor(0.1997)

In [34]:
class QuantileRegressor:
    def __init__(self, quantile, alpha=None):
        self.quantile = quantile

    def fit(self, X, Y, n_iter=100):
        self.dim = X.shape[1]
        self.beta = torch.zeros(self.dim + 1, requires_grad=True, dtype=float)
        X = torch.as_tensor(np.concatenate([X, np.ones((X.shape[0], 1))], axis=1))
        L = max(self.quantile, 1 - self.quantile)
        optim = torch.optim.SGD([self.beta], lr=0.3 / L)
        for i in range(n_iter):
            loss = self.quantile_loss(Y - X @ self.beta, self.quantile).mean()
            loss.backward()
            optim.step()
            optim.zero_grad()
        return self

    def predict(self, X):
        X = torch.as_tensor(np.concatenate([X, np.ones((X.shape[0], 1))], axis=1))
        return X @ self.beta

    @staticmethod
    def quantile_loss(error, q):
        loss = q * torch.relu(error) + (1 - q) * torch.relu(-error)
        return loss

In [35]:
Y = torch.as_tensor(Y)

In [36]:
m = QuantileRegressor(quantile=0.8, alpha=0.0).fit(X, Y)

In [37]:
(Y - m.predict(X) > 0).float().mean()

tensor(0.2000)